In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler
    ]
)


example = [
    {
        "movie":"The God father I",
        "answer": """
        Infomation of The God father
        Director:  Francis Ford Coppola
        Main Cast: Al Pacino, Robert Duvall, Diane Keaton, Robert De Niro
        Budget: $6 million 
        Box Office Revenue: $130 million
        Genre: Mafia movies
        Synopsis: In the village of Corleone, Sicily, Vito Corleone loses his entire family to the local mafia and moves to New York. Armed with his signature business acumen and compassion for the vulnerable, Vito Corleone rises in stature as he resolves the problems of Italian immigrants, both large and small, and becomes a "godfather" to all.
        """
    },
    {
        "movie":"Blade Runner",
        "answer": """
        Infomation of Blade Runner
        Director:  Ridley Scott
        Main Cast: Harrison Ford, Rutger Oelsen Hauer, Sean Young
        Budget: $28 million
        Box Office Revenue: $41,815,801
        Genre: Cyberpunk, SF, Action
        Synopsis: A film directed by Ridley Scott, based on Philip K. Dick's science fiction novel "Do Androids Dream of Electric Sheep?" Originally released in 1982, it was met with critical and commercial acclaim, but later received critical acclaim, earning it the nickname "The Cursed Masterpiece."
        """
    },
    {
        "movie":"Braveheart",
        "answer": """
        Infomation of Braveheart
        Director:  Mel Gibson
        Main Cast: Mel Gibson, Sophie Marceau, Patrick Joseph McGoohan
        Budget: $72 million
        Box Office Revenue: $213,216,216
        Genre: Drama, war, action, historical drama
        Synopsis: This war film, which is considered a masterpiece in film history and is based on the life of William Wallace, a Scottish independence activist and war hero who was active in the First War of Scottish Independence, is directed by Mel Gibson and stars in it.
        """
    },
]

example_prompt = PromptTemplate.from_template(  #prompt의 예시를 만들어 주고
    "human:Tell me about{movie}. \n AI: {answer}"
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt, #Prompt의 예시와 
    example = example, #example을 집어 넣었고
    suffix = "Human: Tell me about {movie} movie", 
    input_variable = ['movie'],
)

chain = prompt | chat
chain.invoke({"movie":"Avata"})


WARNING! callback is not default parameter.
                    callback was transferred to model_kwargs.
                    Please confirm that callback is what you intended.


ValidationError: 1 validation error for FewShotPromptTemplate
__root__
  One of 'examples' and 'example_selector' should be provided (type=value_error)